In [1]:
# Importar bibliotecas necesarias
import pandas as pd
import mysql.connector
import json

In [16]:

# Leer el archivo CSV
df = pd.read_csv('../data/requests.csv', sep=';')

In [17]:
df.columns

Index(['Nº Registro', 'Tipo', 'Estado', 'F.Solicitud', 'Origen', 'Unidad',
       'Provincia', 'Referencia', 'F.Resolucion'],
      dtype='object')

In [15]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random

# Leer el archivo CSV
df = pd.read_csv('../data/requests.csv', sep=';')

# Suponiendo que ya tienes el dataframe con la columna 'F.solicitud' y 'Estado'
df['F.Solicitud'] = pd.to_datetime(df['F.Solicitud'])

# Función para generar F.resolucion
def generar_fecha_resolucion(row):
    if row['Estado'] == 'Contestado':
        # Genera un número aleatorio de días entre 5 y 50
        dias_random = np.random.randint(5, 51)
        # Suma los días a F.solicitud
        return row['F.Solicitud'] + pd.Timedelta(days=dias_random)
    else:
        # Si no está contestado, retorna NaT (Not a Time)
        return pd.NaT

# Aplicar la función al dataframe para generar la columna F.resolucion
df['F.Resolucion'] = df.apply(generar_fecha_resolucion, axis=1)

# Guardar los cambios en el archivo CSV
df.to_csv('../data/requests_fecha.csv', sep=';', index=False)

# Mostrar las primeras filas del DataFrame para verificar
print(df.head())


  Nº Registro                Tipo      Estado F.Solicitud        Origen  \
0   2023-1112               Queja  Contestado  2023-12-30         LIBRO   
1   2023-1109               Queja  Contestado  2023-12-29  Servicio 012   
2   2023-1110               Queja  Contestado  2023-12-29  Servicio 012   
3   2023-1111  Sugerencia Interna  Contestado  2023-12-29           WEB   
4   2023-1108               Queja  Contestado  2023-12-28  Servicio 012   

                              Unidad            Provincia  \
0     GERENCIA DE SERVICIOS SOCIALES              SEGOVIA   
1  CONSEJERIA DE ECONOMIA Y HACIENDA               BURGOS   
2  CONSEJERIA DE ECONOMIA Y HACIENDA               BURGOS   
3              CONSEJERIA DE SANIDAD  SERVICIOS CENTRALES   
4     GERENCIA DE SERVICIOS SOCIALES           VALLADOLID   

                                          Referencia F.Resolucion  
0  El interesado se queja de varias cosas: cenas ...   2024-01-21  
1                                 OAMR horario

In [5]:
import os
print(f"Directorio de trabajo actual: {os.getcwd()}")


Directorio de trabajo actual: /home/mauro/Data-Engineer/python/citizen-requests-analysis/notebook


In [4]:
df.columns

Index(['Nº Registro', 'Tipo', 'Estado', 'F.Solicitud', 'Origen', 'Unidad',
       'Provincia', 'Referencia'],
      dtype='object')

In [5]:
df.drop_duplicates()

,Nº Registro,Tipo,Estado,F.Solicitud,Origen,Unidad,Provincia,Referencia
0,2023-1112,Queja,Contestado,2023-12-30,LIBRO,GERENCIA DE SERVICIOS SOCIALES,SEGOVIA,El interesado se queja de varias cosas: cenas ...
1,2023-1109,Queja,Contestado,2023-12-29,Servicio 012,CONSEJERIA DE ECONOMIA Y HACIENDA,BURGOS,OAMR horario tarde
2,2023-1110,Queja,Contestado,2023-12-29,Servicio 012,CONSEJERIA DE ECONOMIA Y HACIENDA,BURGOS,OAMR horario tarde
3,2023-1111,Sugerencia Interna,Contestado,2023-12-29,WEB,CONSEJERIA DE SANIDAD,SERVICIOS CENTRALES,Análisis SINTROM en SACYL CONECTA
4,2023-1108,Queja,Contestado,2023-12-28,Servicio 012,GERENCIA DE SERVICIOS SOCIALES,VALLADOLID,NaN
...,...,...,...,...,...,...,...,...
4457,2020-0007,Queja,Contestado,2020-01-04,LIBRO,CONSEJERIA DE CULTURA Y TURISMO,SERVICIOS CENTRALES,CALEFACCIÓN AVERIADA
4458,2020-0005,Queja,Contestado,2020-01-04,LIBRO,CONSEJERIA DE CULTURA Y TURISMO,SERVICIOS CENTRALES,FALLOS CALEFACCIÓN
4459,2020-0003,Queja,Contestado,2020-01-03,LIBRO,CONSEJERIA DE EDUCACION Y CULTURA,SEGOVIA,El interesado se queja de la ocupación de los ...
4460,2020-0002,Queja,Contestado,2020-01-03,WEB,CONSEJERIA DE CULTURA Y TURISMO,LEÓN,Progamación clases esqui incumplidas y falta c...


In [6]:
# Conexión a MySQL
DB_CONFIG = {
    'host': 'localhost',
    'user': 'user',
    'password': 'password',
    'database': 'data_db'
}


In [7]:

# Creación de la conexión a MySQL
connection = mysql.connector.connect(
    host=DB_CONFIG['host'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password'],
    database=DB_CONFIG['database']
)
cursor = connection.cursor()

ProgrammingError: 1045 (28000): Access denied for user 'user'@'localhost' (using password: YES)

In [ ]:
create_tables_sql = [
    """
    CREATE TABLE IF NOT EXISTS kpi_volumen_pendientes (
        trimestre INT,
        año INT,
        volumen_pendientes INT,
        total_solicitudes INT,
        porcentaje_pendientes DECIMAL(5, 2),
        objetivo DECIMAL(5, 2) DEFAULT 10,
        PRIMARY KEY (trimestre, año)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS kpi_porcentaje_resueltas (
        trimestre INT,
        año INT,
        porcentaje_resueltas DECIMAL(5, 2),
        objetivo DECIMAL(5, 2) DEFAULT 90,
        PRIMARY KEY (trimestre, año)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS kpi_distribucion_origen (
        trimestre INT,
        año INT,
        origen VARCHAR(50),
        cantidad INT,
        total_solicitudes INT,
        porcentaje DECIMAL(5, 2),
        objetivo DECIMAL(5, 2),
        PRIMARY KEY (trimestre, año, origen)
    )
    """
]

# Ejecutar cada consulta individualmente
for query in create_tables_sql:
    cursor.execute(query)


In [3]:
df['Origen'].unique(

)

array(['LIBRO', 'Servicio 012', 'WEB', 'ESCRITO', 'OAMR', 'BUZON', 'FAX'],
      dtype=object)

In [18]:
df['Estado'].unique(

)

array(['Contestado', 'Prestacional', 'Anulada', 'Remitida Órgano',
       'Tributos', 'Remitido Órgano', 'Inadmitida', 'Remitido a Cons/DT',
       'Rechazado', 'Recibido'], dtype=object)

KPI 1: Volumen de Solicitudes Pendientes

In [9]:
# Asumiendo que tu DataFrame es df
df['F.Solicitud'] = pd.to_datetime(df['F.Solicitud'])

# Agrupar por trimestre y año
df['trimestre'] = df['F.Solicitud'].dt.to_period('Q').dt.quarter
df['año'] = df['F.Solicitud'].dt.year

# KPI 1: Volumen de Solicitudes Pendientes
kpi_volumen = df.groupby(['trimestre', 'año']).agg(
    volumen_pendientes=pd.NamedAgg(column='Estado', aggfunc=lambda x: (x != 'Contestado').sum()),
    total_solicitudes=pd.NamedAgg(column='Estado', aggfunc='count')
).reset_index()

# Calcular porcentaje de pendientes
kpi_volumen['porcentaje_pendientes'] = (kpi_volumen['volumen_pendientes'] / kpi_volumen['total_solicitudes']) * 100

# Establecer el objetivo (porcentaje de pendientes)
kpi_volumen['objetivo'] = 10  # Ejemplo de objetivo: 10%

# Inserción en la tabla kpi_volumen_pendientes
for index, row in kpi_volumen.iterrows():
    cursor.execute("""
    INSERT INTO kpi_volumen_pendientes (trimestre, año, volumen_pendientes, total_solicitudes, porcentaje_pendientes, objetivo)
    VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['trimestre'], row['año'], row['volumen_pendientes'], row['total_solicitudes'], row['porcentaje_pendientes'], row['objetivo']))

connection.commit()

NameError: name 'cursor' is not defined

KPI 2: Porcentaje de Solicitudes Resueltas

In [10]:
# KPI 2: Porcentaje de Solicitudes Resueltas
kpi_resueltas = df.groupby(['trimestre', 'año']).agg(
    porcentaje_resueltas=pd.NamedAgg(column='Estado', aggfunc=lambda x: (x == 'Contestado').sum() / x.count() * 100)
).reset_index()

# Establecer el objetivo (porcentaje de solicitudes resueltas)
kpi_resueltas['objetivo'] = 75  # Ejemplo de objetivo: 75%

# Inserción en la tabla kpi_porcentaje_resueltas
for index, row in kpi_resueltas.iterrows():
    cursor.execute("""
    INSERT INTO kpi_porcentaje_resueltas (trimestre, año, porcentaje_resueltas, objetivo)
    VALUES (%s, %s, %s, %s)
    """, (row['trimestre'], row['año'], row['porcentaje_resueltas'], row['objetivo']))

connection.commit()

NameError: name 'cursor' is not defined

KPI 3: Reducción de Solicitudes por Origen (LIBRO)

In [ ]:
# Paso 1: Calcular la cantidad de solicitudes LIBRO por trimestre y año
kpi_libro = df[df['Origen'] == 'LIBRO'].groupby(['trimestre', 'año']).agg(
    cantidad_libro=pd.NamedAgg(column='Estado', aggfunc='count')
).reset_index()

# Paso 2: Calcular el total de solicitudes por trimestre y año
total_solicitudes_trimestre = df.groupby(['trimestre', 'año']).agg(
    total_solicitudes=pd.NamedAgg(column='Estado', aggfunc='count')
).reset_index()

# Comprobación visual de los datos antes de realizar el merge
print("Cantidad de solicitudes LIBRO por trimestre y año:")
print(kpi_libro)
print("Total de solicitudes por trimestre y año:")
print(total_solicitudes_trimestre)

# Paso 3: Merge para agregar el total de solicitudes al DataFrame kpi_libro
kpi_libro = kpi_libro.merge(total_solicitudes_trimestre, on=['trimestre', 'año'], how='left')

# Paso 4: Calcular el porcentaje de solicitudes LIBRO respecto al total de solicitudes del trimestre
kpi_libro['porcentaje_libro'] = (kpi_libro['cantidad_libro'] / kpi_libro['total_solicitudes']) * 100

# Comprobación visual del porcentaje
print("Porcentaje de solicitudes LIBRO por trimestre y año:")
print(kpi_libro[['trimestre', 'año', 'cantidad_libro', 'total_solicitudes', 'porcentaje_libro']])

# Paso 5: Calcular el porcentaje de LIBRO del trimestre anterior para establecer el objetivo
# Ordenar el DataFrame por 'año' y 'trimestre' antes de aplicar el shift para calcular el objetivo de reducción
kpi_libro = kpi_libro.sort_values(by=['año', 'trimestre'])
kpi_libro['porcentaje_libro_anterior'] = kpi_libro['porcentaje_libro'].shift(1)

# Paso 6: Calcular el objetivo de reducción como un 10% menos que el porcentaje del trimestre anterior
# Si es el primer trimestre del año (no hay valor anterior), el objetivo será 0 por defecto
kpi_libro['objetivo_reduccion'] = kpi_libro['porcentaje_libro_anterior'] * 0.90

# Asegurarse de que no haya valores NaN antes de la inserción, especialmente en la columna 'objetivo_reduccion'
kpi_libro = kpi_libro.fillna(0)  # Reemplazar NaN por 0 u otro valor adecuado según el contexto

# Paso 7: Verificación final de los datos antes de la inserción
print("Datos finales listos para la inserción:")
print(kpi_libro[['trimestre', 'año', 'cantidad_libro', 'total_solicitudes', 'porcentaje_libro', 'porcentaje_libro_anterior', 'objetivo_reduccion']])

# Paso 8: Insertar los datos en la tabla kpi_distribucion_origen
for index, row in kpi_libro.iterrows():
    cursor.execute("""
    INSERT INTO kpi_distribucion_origen (trimestre, año, origen, cantidad, total_solicitudes, porcentaje, objetivo)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        int(row['trimestre']),
        int(row['año']),
        'LIBRO',  # El origen es LIBRO, ya que es lo que se está midiendo
        int(row['cantidad_libro']),  # Cantidad de solicitudes LIBRO en ese trimestre
        int(row['total_solicitudes']),  # Total de solicitudes en ese trimestre
        float(row['porcentaje_libro']),  # Porcentaje de solicitudes LIBRO respecto al total
        float(row['objetivo_reduccion'])  # Objetivo de reducción para el siguiente trimestre
    ))

# Confirmar las transacciones y cerrar la conexión
connection.commit()
print("Datos insertados en la tabla kpi_distribucion_origen correctamente.")


Cantidad de solicitudes LIBRO por trimestre y año:
    trimestre   año  cantidad_libro
0           1  2020              32
1           1  2021              21
2           1  2022              59
3           1  2023              13
4           2  2020               2
5           2  2021              39
6           2  2022              39
7           2  2023              22
8           3  2020              48
9           3  2021              33
10          3  2022              26
11          3  2023              17
12          4  2020              33
13          4  2021              34
14          4  2022              31
15          4  2023              10
Total de solicitudes por trimestre y año:
    trimestre   año  total_solicitudes
0           1  2020                177
1           1  2021                327
2           1  2022                325
3           1  2023                284
4           2  2020                209
5           2  2021                268
6           2  2022   

In [ ]:
df['Origen'].value_counts() #Guardar para un notebook de exploracion

Origen
WEB             2382
Servicio 012     954
ESCRITO          465
LIBRO            459
OAMR             175
BUZON             26
FAX                1
Name: count, dtype: int64

In [ ]:
# kpi_volumen, kpi_resueltas, kpi_libro
# Crear columna adicional 'año_trimestre' para usar en los gráficos
kpi_volumen['año_trimestre'] = kpi_volumen['año'].astype(str) + "-Q" + kpi_volumen['trimestre'].astype(str)
kpi_resueltas['año_trimestre'] = kpi_resueltas['año'].astype(str) + "-Q" + kpi_resueltas['trimestre'].astype(str)
kpi_libro['año_trimestre'] = kpi_libro['año'].astype(str) + "-Q" + kpi_libro['trimestre'].astype(str)

# 1. Generar el archivo JSON para 'kpi_volumen_pendientes'
kpi_volumen_json = kpi_volumen[['trimestre', 'año', 'año_trimestre', 'volumen_pendientes', 'objetivo']]
kpi_volumen_json.to_json('../amcharts/kpi_volumen_pendientes.json', orient='records', force_ascii=False)

print("Archivo kpi_volumen_pendientes.json creado exitosamente.")

# 2. Generar el archivo JSON para 'kpi_porcentaje_resueltas'
kpi_resueltas_json = kpi_resueltas[['trimestre', 'año', 'año_trimestre', 'porcentaje_resueltas', 'objetivo']]
kpi_resueltas_json.to_json('../amcharts/kpi_porcentaje_resueltas.json', orient='records', force_ascii=False)


print("Archivo kpi_porcentaje_resueltas.json creado exitosamente.")

# 3. Generar el archivo JSON para 'kpi_distribucion_origen'
kpi_libro_json = kpi_libro[['trimestre', 'año', 'año_trimestre', 'cantidad_libro', 'total_solicitudes', 'porcentaje_libro', 'objetivo_reduccion']]
kpi_libro_json.columns = ['trimestre', 'año', 'año_trimestre', 'cantidad', 'total_solicitudes', 'porcentaje', 'objetivo']
kpi_libro_json.to_json('../amcharts/kpi_distribucion_origen.json', orient='records', force_ascii=False)


print("Archivo kpi_distribucion_origen.json creado exitosamente.")


Archivo kpi_volumen_pendientes.json creado exitosamente.
Archivo kpi_porcentaje_resueltas.json creado exitosamente.
Archivo kpi_distribucion_origen.json creado exitosamente.


In [ ]:
# Confirmar las transacciones y cerrar la conexión
connection.commit()
cursor.close()
connection.close()
print("Tablas de KPIs creadas con columnas adicionales y datos cargados en MySQL correctamente.")

Tablas de KPIs creadas con columnas adicionales y datos cargados en MySQL correctamente.
